In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Looks for .env in the current folder
openai_api_key = os.getenv("OPENAI_API_KEY")
print(openai_api_key)  # This should now print your actual key!


In [ ]:
%pip install tiktoken

from dotenv import load_dotenv
import os


from langchain_community.document_loaders import PyPDFLoader

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

loader = PyPDFLoader("data/who_guidelines.pdf")  # ✅ Use raw or double-backslash path
docs = loader.load()


# Break it into small parts
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# ✅ Pass API key correctly here
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Build the vector store
vectorstore = FAISS.from_documents(chunks, embeddings)

# Smart brain
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, openai_api_key=openai_api_key),
    chain_type="stuff",
    retriever=retriever
)

# Chat function
def health_chatbot():
    print("👩‍⚕️ Hello! Ask me about health (type 'exit' to quit).")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            break
        answer = qa_chain.run(question)
        print("Bot:", answer)

health_chatbot()
